In [1]:
import json
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import pickle
from scipy.stats import multivariate_normal
from sklearn.model_selection import KFold
from sklearn import mixture
from sklearn.decomposition import PCA
from time import time
from tqdm import tqdm

%matplotlib notebook

# http://pypr.sourceforge.net/mog.html

In [2]:
data_path = './db.npy'
classification_path = './classification.csv'
model_dir = './models/'

n_components = 6
with open('config.json') as f:
    config = json.load(f)

config['gmm_components'] = n_components

with open('config.json', 'w+') as f:
    json.dump(config, f)

### Data

In [3]:
data = np.load(data_path).item()
data_gmm = np.hstack([data['spec_features'], data['ts_features']])
data_ts = data['ts_features']
data_spectra = data['spec_features']
dim_ts = data['ts_features'].shape[-1]
dim_spec = data['spec_features'].shape[-1]

In [4]:
probs = pd.read_csv(classification_path)
probs.head()

,EA_ts,EW_ts,RRab_ts,RRc_ts,RRd_ts,RS CVn_ts,EA_mix,EW_mix,RRab_mix,RRc_mix,...,rf_index_mix,target,ts_names,spec_names,pred_ts,pred_ts_prob,ground_ts_prob,pred_mix,pred_mix_prob,ground_mix_prob
0,0.0,0.030,0.005,0.945,0.020,0.0,0.000,0.030,0.000,0.970,...,0,RRc,1132062052528,spec-2112-53534-0521.fits,RRc,0.945,0.945,RRc,0.970,0.970
1,0.0,0.950,0.005,0.040,0.005,0.0,0.005,0.965,0.015,0.015,...,0,EW,1001072050924,spec-3307-54970-0075.fits,EW,0.950,0.950,EW,0.965,0.965
2,0.0,0.005,0.985,0.010,0.000,0.0,0.000,0.000,0.995,0.000,...,0,RRab,1121054053106,spec-2477-54058-0261.fits,RRab,0.985,0.985,RRab,0.995,0.995
3,0.0,0.035,0.025,0.925,0.015,0.0,0.000,0.050,0.005,0.930,...,0,RRc,1126077024700,spec-2474-54564-0169.fits,RRc,0.925,0.925,RRc,0.930,0.930
4,0.0,0.040,0.000,0.935,0.025,0.0,0.000,0.080,0.000,0.815,...,0,RRc,1138033064612,spec-3661-55614-0596.fits,RRc,0.935,0.935,RRc,0.815,0.815


### Conditional GMM

In [5]:
class GMM:
    def __init__(self, mean, cov, pi):
        self.mean = np.array(mean)
        self.cov = np.array(cov)
        self.pi = np.array(pi)
        
    def random_samples(self, size):
        sample_idx = np.random.choice(np.arange(self.pi.shape[0]), size=size, p=self.pi)
        samples = np.array([np.random.multivariate_normal(self.mean[i], self.cov[i]) for i in sample_idx])
        return samples
    
    def pdf(self,x):
        x = np.array(x)
        if x.ndim == 1:
            x = np.expand_dims(x, axis=0)
        pdf_gmm_func = lambda x: [multivariate_normal.pdf(x, mean=self.mean[i], cov=self.cov[i]) for i in range(len(self.pi))]
        pdf_gmm = np.array(list(map(pdf_gmm_func, x)))
        pdf_x = np.sum(self.pi*pdf_gmm, axis=1)
        return pdf_x
    
    def entropy(self, x):
        p = self.pdf(x)
        C = np.sum(p)
        p /= C
        x_entropy = -np.sum(p*np.log(p))
        return x_entropy

def gauss_cond(u,cov):
    u1 = u[:dim_spec]
    u2 = u[dim_spec:]
    cov11 = cov[:dim_spec, :dim_spec]
    cov12 = cov[:dim_spec, dim_spec:]
    cov21 = cov[dim_spec:, :dim_spec]
    cov22 = cov[dim_spec:, dim_spec:]
    u_cond_base = np.matmul(cov12, np.linalg.inv(cov22))
    u_cond = lambda Y2: u1 + np.matmul(u_cond_base, Y2-u2)
    cov_cond = cov11 - np.matmul(np.matmul(cov12, np.linalg.inv(cov22)), cov21)
    params_cond = lambda Y: (u_cond(Y), cov_cond)
    return params_cond

def gmm_cond(ts, gmm):
    cond = list(map(lambda x: gauss_cond(*x), zip(gmm.means_, gmm.covariances_)))
    cond_params = [gauss(ts) for gauss in cond]
    cond_params = list(zip(*cond_params))
    cond_u = np.array(cond_params[0])
    cond_cov = np.array(cond_params[1])
    
    min_ = 1e-15
    ts_u = gmm.means_[:, -dim_ts:]
    ts_cov = gmm.covariances_[:, -dim_ts:, -dim_ts:]
    ts_pdf = np.array([multivariate_normal.pdf(ts, mean=ts_u[i], cov=ts_cov[i]) for i in range(len(ts_u))])
    ts_pdf[ts_pdf<min_] = min_
    cond_pi = gmm.weights_*ts_pdf/np.sum(gmm.weights_*ts_pdf)
    return GMM(cond_u, cond_cov, cond_pi)

In [6]:
def loglikelihood_rate(data_ts, data_spectra, n_components):
    
    ll_all = []
    ll_base_all = []
    ll_rate = []
    
    kf = KFold(n_splits=10, shuffle=True)
    for train_idx, test_idx in kf.split(data_ts):
        train_ts = data_ts[train_idx]
        train_spec = data_spectra[train_idx]
        train_data = np.hstack([train_spec, train_ts])
        test_ts = data_ts[test_idx]
        test_spec = data_spectra[test_idx]
        
        gmm = mixture.BayesianGaussianMixture(n_components=n_components, max_iter=200).fit(train_data)

        ll = []
        for ts, spec in zip(test_ts, test_spec):
            gmm_ = gmm_cond(ts, gmm)
            ll_i = gmm_.pdf(spec)
            ll.append(ll_i)
        ll = np.sum(np.log(ll))

        mean0 = np.full(test_spec.shape[-1], 0)
        cov1 = np.identity(test_spec.shape[-1])
        ll_base = np.array(multivariate_normal.pdf(test_spec, mean=mean0, cov=cov1))
        ll_base = np.sum(np.log(ll_base))
        
        ll_all.append(ll)
        ll_base_all.append(ll_base)
        ll_rate.append(ll-ll_base)
    
    return ll_all, ll_base_all, ll_rate

In [7]:
# ll_all, ll_base_all, ll_rate = loglikelihood_rate(data_ts, data_spectra, n_components)
# print('Loglikelihood rate mean:', np.mean(ll_rate))
# print('Loglikelihood rate std:', np.std(ll_rate))
# print('Loglikelihood rates:', ll_rate)

In [8]:
def get_gmm(data_gmm):
    best_ll = None
    gmm = None
    for i in range(10):
        t0 = time()
        gmm_i = mixture.BayesianGaussianMixture(n_components=n_components, max_iter=400).fit(data_gmm)
        t1 = np.round(time()-t0, decimals=2)
        msg = 'Time '+str(t1)+'.s'
        print(msg)
        gmm_aux = GMM(gmm_i.means_, gmm_i.covariances_, gmm_i.weights_)
        ll = np.sum(np.log(gmm_aux.pdf(data_gmm)))

        if gmm is None:
            gmm = gmm_i
            best_ll = ll
        elif best_ll < ll:
            gmm = gmm_i
            best_ll = ll
            
    print('Best Loglikelihood: ', best_ll)
    return gmm

In [9]:
gmm = get_gmm(data_gmm)

Time 0.28.s
Time 0.1.s
Time 0.59.s
Time 0.64.s
Time 0.31.s
Time 0.1.s
Time 0.23.s
Time 0.3.s
Time 0.1.s
Time 0.14.s
Best Loglikelihood:  41548.25348859327


### Average probabilities with sampled spectra

In [10]:
def add_marginal(probs, data, gmm, model_dir, size=200):
    probs = probs.set_index('ts_names', drop=True)
    key_ = 'ts_names'
    keys = data[key_]
    
    marginal = []
    H_mean = []
    H_fn = lambda x: -np.sum(x*np.log(x))
    H_spec = []
    
    for i, key in tqdm(enumerate(keys)):
        clf_index = probs.loc[key]['rf_index_mix']
        clf_path = model_dir+'mix_rf'+str(clf_index)+'.model'
        clf = pickle.load(open(clf_path, 'rb'))
        
        ts = data['ts_features'][i]
        ts_tile = np.tile(ts, [size,1])
        dist = gmm_cond(ts, gmm)
        samples = dist.random_samples(size)
        samples_p = clf.predict_proba(np.hstack([ts_tile, samples]))
        samples_p[samples_p==0.0] = 1e-10
        samples_pred = np.argmax(samples_p, axis=-1)
        
        cls, counts = np.unique(samples_pred, return_counts=True)
        cls = clf.classes_[cls]
        counts = counts/sum(counts)
        marginal.append(dict(zip(cls, counts)))
        
        H_mean_i = np.apply_along_axis(H_fn, axis=-1, arr=samples_p)
        H_mean.append(np.mean(H_mean_i))
        
        H_spec_i = dist.entropy(samples)
        H_spec.append(H_spec_i)
    
    marginal = pd.DataFrame(marginal)
    marginal = marginal.rename(lambda col: col+'_marg', axis=1)
    marginal[key_] = keys
    marginal = marginal.set_index(key_, drop=True)
    marginal = marginal.fillna(0)
    
    H_mean = pd.DataFrame(H_mean, columns=['H_mix_mean'])
    H_mean[key_] = keys
    H_mean = H_mean.set_index(key_, drop=True)
    
    H_spec = pd.DataFrame(H_spec, columns=['s_uncertainty'])
    H_spec[key_] = keys
    H_spec = H_spec.set_index(key_, drop=True)
    
    probs = pd.concat([probs, marginal, H_mean, H_spec], axis=1)
    assert probs.isna().sum().sum() == 0, 'NaN values'
    
    prefix = np.unique(probs['target'])
    cols_marg = prefix + '_marg'
    idx = probs.index
    fn = lambda str_: str_.rstrip('_marg')
    probs['pred_marg'] = probs[cols_marg].idxmax(axis=1)
    probs['pred_marg_prob'] = probs.lookup(idx,probs['pred_marg'])
    probs['pred_marg'] = probs['pred_marg'].apply(fn)
    
    return probs

def add_entropy(probs):
    prefix = np.unique(probs['target'])
    cols_ts =  prefix + '_ts'
    cols_mix = prefix + '_mix'
    cols_marg = prefix + '_marg'
    entropy_fn = lambda x: -np.sum(x*np.log(x))
    probs['H_ts'] = probs[cols_ts].apply(entropy_fn,axis=1)
    probs['H_mix'] = probs[cols_mix].apply(entropy_fn,axis=1)
    probs['H_diff_approx'] = probs['H_ts'] - probs['H_mix_mean']
    probs['ground_diff'] = probs['ground_mix_prob'] - probs['ground_ts_prob']
    assert probs.isna().sum().sum() == 0, 'NaN values'
    
    return probs

In [11]:
probs = add_marginal(probs, data, gmm, model_dir, 200)
probs.head()

2554it [06:43,  6.43it/s]


,EA_ts,EW_ts,RRab_ts,RRc_ts,RRd_ts,RS CVn_ts,EA_mix,EW_mix,RRab_mix,RRc_mix,...,EA_marg,EW_marg,RRab_marg,RRc_marg,RRd_marg,RS CVn_marg,H_mix_mean,s_uncertainty,pred_marg,pred_marg_prob
ts_names,,,,,,,,,,,,,,,,,,,,,
1132062052528,0.0,0.030,0.005,0.945,0.020,0.0,0.000,0.030,0.000,0.970,...,0.0,0.000,0.0,1.000,0.0,0.0,0.187479,5.070082,RRc,1.000
1001072050924,0.0,0.950,0.005,0.040,0.005,0.0,0.005,0.965,0.015,0.015,...,0.0,1.000,0.0,0.000,0.0,0.0,0.164062,5.136999,EW,1.000
1121054053106,0.0,0.005,0.985,0.010,0.000,0.0,0.000,0.000,0.995,0.000,...,0.0,0.000,1.0,0.000,0.0,0.0,0.220362,5.054955,RRab,1.000
1126077024700,0.0,0.035,0.025,0.925,0.015,0.0,0.000,0.050,0.005,0.930,...,0.0,0.045,0.0,0.955,0.0,0.0,0.309486,4.947624,RRc,0.955
1138033064612,0.0,0.040,0.000,0.935,0.025,0.0,0.000,0.080,0.000,0.815,...,0.0,0.005,0.0,0.995,0.0,0.0,0.169345,5.042658,RRc,0.995


In [12]:
probs = add_entropy(probs)
probs.head()

/Users/Javiera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in log


,EA_ts,EW_ts,RRab_ts,RRc_ts,RRd_ts,RS CVn_ts,EA_mix,EW_mix,RRab_mix,RRc_mix,...,RRd_marg,RS CVn_marg,H_mix_mean,s_uncertainty,pred_marg,pred_marg_prob,H_ts,H_mix,H_diff_approx,ground_diff
ts_names,,,,,,,,,,,,,,,,,,,,,
1132062052528,0.0,0.030,0.005,0.945,0.020,0.0,0.000,0.030,0.000,0.970,...,0.0,0.0,0.187479,5.070082,RRc,1.000,0.263388,0.134742,0.075909,0.025
1001072050924,0.0,0.950,0.005,0.040,0.005,0.0,0.005,0.965,0.015,0.015,...,0.0,0.0,0.164062,5.136999,EW,1.000,0.230467,0.186863,0.066405,0.015
1121054053106,0.0,0.005,0.985,0.010,0.000,0.0,0.000,0.000,0.995,0.000,...,0.0,0.0,0.220362,5.054955,RRab,1.000,0.087430,0.031479,-0.132932,0.010
1126077024700,0.0,0.035,0.025,0.925,0.015,0.0,0.000,0.050,0.005,0.930,...,0.0,0.0,0.309486,4.947624,RRc,0.955,0.344666,0.306765,0.035180,0.005
1138033064612,0.0,0.040,0.000,0.935,0.025,0.0,0.000,0.080,0.000,0.815,...,0.0,0.0,0.169345,5.042658,RRc,0.995,0.283817,0.605429,0.114472,-0.120


In [13]:
probs.to_csv('classification_entropy.csv')